## 1. Import data
Adapted from: http://rnduja.github.io/2015/10/13/torch-mnist/

We use data from the standard MNIST set

In [1]:
require 'torch'
require 'nn'
require 'cunn'
require 'cudnn'
require 'optim'
mnist = require 'mnist'

backend = cudnn

In [2]:
fullset = mnist.traindataset()
testset = mnist.testdataset()

In [3]:
fullset

{
  data : ByteTensor - size: 60000x28x28
  size : 60000
  label : ByteTensor - size: 60000
}


We inspect the data just to get an idea of the content

In [4]:
itorch.image(fullset.data[1])

In [5]:
fullset.label[1]

5	


We can split the full dataset into a trainin component and a validation component, which will be used to train hyperparameters.

While doing so, we convert the dataset to double

In [6]:
trainset = {
    size = 50000,
    data = fullset.data[{{1,50000}}]:double(),
    label = fullset.label[{{1,50000}}]
}
trainset.data = trainset.data:cuda()

In [7]:
validationset = {
    size = 10000,
    data = fullset.data[{{50001,60000}}]:double(),
    label = fullset.label[{{50001,60000}}]
}
validationset.data = validationset.data:cuda()

## 2. Create the model

We use a model with a single hidden layer, using a hyperbolic tangent activation, and a softmax output. We also use a first layer to reshape the input - which is a 28x28 square - to fit into the linear layer

In [8]:
model = nn.Sequential()
model:add(nn.Reshape(28*28))
model:add(nn.Linear(28*28, 30))
model:add(nn.Tanh())
model:add(nn.Linear(30, 10))
model:add(nn.LogSoftMax())

In [8]:
--This is a LeNet model. For more information: http://yann.lecun.com/exdb/lenet/

model = nn.Sequential()
model:add(nn.MulConstant(0.0125)) -- 1/(standard deviation on MNIST dataset)
model:add(backend.SpatialConvolution(1,20,5,5,1,1,0)) -- channels*28*28 -> 20*24*24
model:add(backend.SpatialMaxPooling(2, 2, 2, 2)) -- 20*24*24 -> 20*12*12
model:add(backend.SpatialConvolution(20,50,5,5,1,1,0)) -- 20*12*12 -> 50*8*8
model:add(backend.SpatialMaxPooling(2,2,2,2)) --  50*8*8 -> 50*4*4
model:add(nn.View(-1):setNumInputDims(3))  -- 50*4*4 -> 800
model:add(nn.Linear(800,500))  -- 800 -> 500
model:add(backend.ReLU())
model:add(nn.Linear(500, 10))  -- 500 -> nclasses
model:add(nn.LogSoftMax())


In [9]:
model = model:cuda()

We also define a loss function, using the negative log likelihood criterion

In [10]:
criterion = nn.ClassNLLCriterion()
criterion = criterion:cuda()

As explained in [the documentation](https://github.com/torch/nn/blob/master/doc/criterion.md), the NLL criterion require the output of the neural network to contain log-probabilities of each class, and this is the reason for our use of `LogSoftMax` above.

## 3. Define the descent algorithm

We will make use of the `optim` package to train the network. `optim` contains several optimization algorithms. All of these algorithms assume the same parameters:

- a closure that computes the loss, and its gradient wrt to `x`, given a point `x`
- a point x
- some parameters, which are algorithm-specific

We define a `step` function that performs training for a single epoch and returns the current loss value

In [11]:
sgd_params = {
   learningRate = 1e-2,
   learningRateDecay = 1e-4,
   weightDecay = 1e-3,
   momentum = 1e-4
}

In [12]:
x, dl_dx = model:getParameters()

In [13]:
step = function(batch_size)
    local current_loss = 0
    local count = 0
    local shuffle = torch.randperm(trainset.size)
    batch_size = batch_size or 32
    
    for t = 1,trainset.size,batch_size do
        -- setup inputs and targets for this mini-batch
        local size = math.min(t + batch_size - 1, trainset.size) - t
        local inputs = torch.CudaTensor(size, 1, 28, 28)
        local targets = torch.CudaTensor(size)
  
        for i = 1,size do
            local input = trainset.data[shuffle[i+t]]
            local target = trainset.label[shuffle[i+t]]
            -- if target == 0 then target = 10 end
            inputs[i] = input
            targets[i] = target
        end
        targets:add(1)
        
        local feval = function(x_new)
            -- reset data
            if x ~= x_new then x:copy(x_new) end
            dl_dx:zero()

            -- perform mini-batch gradient descent
            local loss = criterion:forward(model:forward(inputs), targets)
            model:backward(inputs, criterion:backward(model.output, targets))

            return loss, dl_dx
        end
        
        _, fs = optim.sgd(feval, x, sgd_params)
        -- fs is a table containing value of the loss function
        -- (just 1 value for the SGD optimization)
        count = count + 1
        current_loss = current_loss + fs[1]
    end

    -- normalize loss
    return current_loss / count
end

Before starting the training, we also need to be able to evaluate accuracy on a separate dataset, in order to define when to stop

In [14]:
eval = function(dataset, batch_size)
    local count = 0
    batch_size = batch_size or 32
    
    for i = 1,dataset.size,batch_size do
        local size = math.min(i + batch_size - 1, dataset.size) - i
        local inputs = dataset.data[{{i,i+size-1}}]:reshape(size,1,28,28):cuda()
        local targets = dataset.label[{{i,i+size-1}}]:cudaLong()
        local outputs = model:forward(inputs)
        local _, indices = torch.max(outputs, 2)
        indices:add(-1)
        local guessed_right = indices:eq(targets):sum()
        count = count + guessed_right
    end

    return count / dataset.size
end

## 4. Train the model

We are now ready to perform the actual training. After each epoch, we evaluate the accuracy on the validation dataset, in order to decide whether to stop

In [15]:
max_iters = 12

In [16]:
do
    local last_accuracy = 0
    local decreasing = 0
    local threshold = 1 -- how many deacreasing epochs we allow
    for i = 1,max_iters do
        local loss = step()
        print(string.format('Epoch: %d Current loss: %4f', i, loss))
        local accuracy = eval(validationset)
        print(string.format('Accuracy on the validation set: %4f', accuracy))
        if accuracy < last_accuracy then
            if decreasing > threshold then break end
            decreasing = decreasing + 1
        else
            decreasing = 0
        end
        last_accuracy = accuracy
    end
end

Epoch: 1 Current loss: 0.349721	


Accuracy on the validation set: 0.936300	


Epoch: 2 Current loss: 0.107826	


Accuracy on the validation set: 0.937300	


Epoch: 3 Current loss: 0.076105	


...

Accuracy on the validation set: 0.953300	


Epoch: 11 Current loss: 0.028584	


Accuracy on the validation set: 0.954900	


Epoch: 12 Current loss: 0.026379	


Accuracy on the validation set: 0.954600	


Let us test the model accuracy on the test set

In [17]:
--testset.data = testset.data:double()
testset.data = testset.data:cuda()

In [18]:
eval(testset)

0.9577	


## 5. Saving and restoring the model

The `paths` module can be used to manipulate filesystem paths

In [19]:
paths = require 'paths'

In [20]:
filename = paths.concat(paths.cwd(), 'model.net')

In [21]:
filename

/home/avelino/torch-tutorials/model.net	


We can then save our model to file like this

In [62]:
help(torch.save)

+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++	
torch.save(filename, object [, format, referenced])


Writes object into a file named filename . The format can be set 
to ascii or binary (default is binary). Binary format is platform
dependent, but typically more compact and faster to read/write. The ASCII
format is platform-independent, and should be used to share data structures
across platforms. The option referenced specifies if
object references should be tracked or not
( true by default).
 -- arbitrary object:
obj = {
   mat = torch.randn(10,10),
   name = '10',
   test = {
      entry = 1
   }
}
-- save to disk:
torch.save('test.dat', obj) 	
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++	



In [63]:
torch.save(filename, model)

Let us check that restoring from file works as expected

In [64]:
help(torch.load)

+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++	
[object] torch.load(filename [, format, referenced])


Reads object from a file named filename .
The format can be set to ascii , binary , b32 or b64 (default 
is binary).
Binary format is platform dependent, but typically more compact and faster 
to read/write.
Use b32 / b64 , instead of binary , for loading files saved on a 
32/64 bit OS.
The ASCII format is platform-independent, and may be used to share data 
structures across platforms.
The option referenced specifies if object references should be tracked 
or not ( true by default).
Note that files written with referenced at true cannot be loaded with 
referenced at false .
 -- given serialized object from section above, reload:
obj = torch.load('test.dat')
print(obj)
-- will print:
-- {[mat]  = DoubleTensor - size: 10x10
--  [name] = string : &quot;10&quot;
--  [test] = table - size: 0} 	
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++	



In [22]:
model1 = torch.load(filename)

We redefine our evaluation function to use the loaded model

In [23]:
eval1 = function(dataset)
   local count = 0
   for i = 1,dataset.size do
      local output = model1:forward(dataset.data[i])
      local _, index = torch.max(output, 1) -- max index
      local digit = index[1] - 1
      if digit == dataset.label[i] then count = count + 1 end
   end

   return count / dataset.size
end

In [25]:
eval(testset)

0.9577	


In [26]:
itorch.image(model.modules[2].weight)